In [ ]:
# obtain required packages
!pip install python-dotenv
!wget -P . https://files.pythonhosted.org/packages/cc/7a/240ac474f69a58a5dda0513557fe9c2ada279f7d5d836b93d5f6e05a9aa7/Mastodon.py-1.8.1.tar.gz 

In [ ]:
# install Mastodon library for conda env
conda install Mastodon.py-1.8.1.tar.gz

In [1]:
# Import required packages
from dotenv import load_dotenv
from mastodon import Mastodon
import os

from bs4 import BeautifulSoup as bs4
import requests

# Load variables from .env file
load_dotenv()

True

In [2]:
# obtain access token from Mastodon
ACCESS_TOKEN = os.environ.get("MASTODON_ACCESS_TOKEN")

# define base URL
BASE_URL = "https://mastodon.social"

In [3]:
# create Mastodon client object
mastodon = Mastodon(api_base_url=BASE_URL, access_token=ACCESS_TOKEN)
mastodon.retrieve_mastodon_version()

'4.1.2'

In [4]:
# sample scraper: use mastodon api to obtain content from a post, then parse it with BeautifulSoup
bs4(mastodon.status("110284900191177710")["content"], 'html.parser').text

'A history of the top marginal tax rates on the wealthiest Americans: 1940: 81%1950: 84% 1960: 91% 1970: 72% 1980: 70% 1990: 28% 2000: 40% 2010: 35% For 50 years, corporate backed politicians in Congress have slashed taxes to line the pockets of their wealthy donors.'

In [5]:
# Import required packages
from mastodon import Mastodon, MastodonNotFoundError, MastodonRatelimitError, StreamListener
import csv, os, time, json

# defien Mastodon client object
m = Mastodon(
        api_base_url=BASE_URL,
        access_token=ACCESS_TOKEN
    )

# Define a Mastodon Listener class
# This listener class will print out the JSON of any updates that come through the stream
# Ti will be an infinite loop, so you will need to stop the program or kernel to stop it
class Listener(StreamListener):
    def on_update(self, status):
        print(json.dumps(status, indent=2, sort_keys=True, default=str))

m.stream_public(Listener())

{
  "account": {
    "acct": "Leks@mastodont.cat",
    "avatar": "https://files.mastodon.social/cache/accounts/avatars/109/298/478/733/993/696/original/509ee4c6e42ef29a.jpg",
    "avatar_static": "https://files.mastodon.social/cache/accounts/avatars/109/298/478/733/993/696/original/509ee4c6e42ef29a.jpg",
    "bot": false,
    "created_at": "2022-11-06 00:00:00+00:00",
    "discoverable": true,
    "display_name": "Leks",
    "emojis": [],
    "fields": [
      {
        "name": "XMPP",
        "value": "leks@mastodont.cat",
        "verified_at": null
      }
    ],
    "followers_count": 87,
    "following_count": 139,
    "group": false,
    "header": "https://files.mastodon.social/cache/accounts/headers/109/298/478/733/993/696/original/fbd8e1de7585481c.jpg",
    "header_static": "https://files.mastodon.social/cache/accounts/headers/109/298/478/733/993/696/original/fbd8e1de7585481c.jpg",
    "id": 109298478733993696,
    "last_status_at": "2023-04-30 00:00:00",
    "locked": false,
 

KeyboardInterrupt: 